In [51]:
# import apriltag
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
import glob
from scipy.optimize import minimize, least_squares
from scipy.spatial.transform import Rotation as scipy_R
import random
import os

In [52]:
%matplotlib inline

# APRILTAG CORNER DEBUG IMAGE GENERATION

In [83]:
serial_number = '22240376'
serial_number = '22206080'
serial_number = '22240368'
serial_number = '22240389'
serial_number = '22222961'
serial_number = '22240382'

checker_board_size = (9, 10) # (rows, cols)

# apriltag_one_side_length = 48.3

render_debug = True

data_root = '/home/hict/yeonkunlee/MULTIVIEW_CAL_HICT/cal_image/intrinsic_images_checker'
debug_dir = os.path.join(data_root, serial_number + '_intrinsic_debug')

# detector = apriltag.Detector()

if not os.path.exists(debug_dir):
    os.makedirs(debug_dir)

images = glob.glob(os.path.join(data_root, serial_number, '*.png'))
images.sort()

print(len(images))

frame = 0
# detector rescale factor is, for preventing apriltag error. apriltag error is caused by the detector's high resolution.
vis_rescale = 0.5

for fname in images:
    img = cv2.imread(fname)
    # result = detector.detect(img)
    # find checkerboard
    success, corners = cv2.findChessboardCorners(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), checker_board_size, flags=cv2.CALIB_CB_FAST_CHECK)

    if success:
        cv2.drawChessboardCorners(img, checker_board_size, corners, success)

    img_plot = cv2.resize(img, (0,0), fx=vis_rescale, fy=vis_rescale)
    
    if render_debug:
        cv2.imwrite(os.path.join(debug_dir,'%04d.jpg'%frame), img_plot)
        frame += 1
    

92


# INTRINSIC_CAL

In [84]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((checker_board_size[0]*checker_board_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:checker_board_size[0],0:checker_board_size[1]].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [85]:
success_count = 0

for fname in images:
    img = cv2.imread(fname)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # result = detector.detect(img)
    # find checkerboard
    success, corners = cv2.findChessboardCorners(img_gray, checker_board_size, flags=cv2.CALIB_CB_FAST_CHECK)

    if success:
        objpoints.append(objp)
        imgpoints.append(corners)
        success_count += 1

print('success_count:', success_count)
print('total images:', len(images))

success_count: 92
total images: 92


In [86]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_gray.shape[::-1], None, None)

In [87]:
print('ret:', ret)
print('mtx:', mtx)
print('dist:', dist)

ret: 0.9717480415469572
mtx: [[1.04768481e+03 0.00000000e+00 9.74139966e+02]
 [0.00000000e+00 1.04668180e+03 5.49169554e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist: [[-0.06147392  0.12850172  0.00191466  0.00038944 -0.06670741]]


In [88]:
# save npz
np.savez(os.path.join('/home/hict/yeonkunlee/MULTIVIEW_CAL_HICT/cal_image', serial_number + '_intrinsic_result.npz'), ret=ret, mtx=mtx, dist=dist)